In [20]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import expit
from scipy.optimize import curve_fit


In [21]:
def step_sim(step_size = 8.0, SNR = 0.5, total_time_point=400):
    np.random.seed(20181118)
    
    steps = np.piecewise(np.arange(0, total_time_point), 
                            [
                                np.arange(0, total_time_point) < 99, 

                                np.logical_and(np.arange(0, total_time_point) >= 99, np.arange(0, total_time_point) < 199), 

                                np.logical_and(np.arange(0, total_time_point) >= 199, np.arange(0, total_time_point) < 249), 
                                
                                np.logical_and(np.arange(0, total_time_point) >= 249, np.arange(0, total_time_point) < 349), 
                                
                                np.logical_and(np.arange(0, total_time_point) >= 349, np.arange(0, total_time_point) < 499), 

                                np.arange(0, total_time_point) >= 499
                            ], 
                            [
                                lambda x: 300, 
                                lambda x: 100, 
                                lambda x: 50, 
                                lambda x: 350,
                                lambda x: 20,
                                lambda x: 350,
                            ])
    x = np.arange(0, total_time_point)
    noise_STD = step_size / SNR
    noise = np.random.normal(scale=noise_STD, size=len(x))
    # y = true_fct + np.random.standard_t(1, size=len(x))
    y = steps + noise
    return x, y, steps



x_data, y_data , steps = step_sim(total_time_point=600)
plt.plot(x_data, y_data, '-b', label="noise")
plt.plot(x_data, steps, '-r', linewidth=1, label="raw")
plt.grid()

In [22]:
from scipy.optimize import curve_fit
from dataclasses import dataclass

@dataclass(repr=True)
class StepInfo:
    step_size:float
    location:int
    stepness:float
    b:float
    
        
# Heaviside step function
# def step_fun(x,a,b,c): return a * (np.sign(x-b) + c)
# p0=[np.max(y), np.median(x), np.min(y)]

def _step_fun(x, step_size ,location, stepness, b):
    # Logistic function
    # f(X, L, x0, k, b)
    # https://en.wikipedia.org/wiki/Logistic_function
    # use for curve fitting
    y = step_size / (1 + np.exp(-stepness*(x-location))) + b
    return (y)

def get_step_location_size(x, y):
    p0=[np.max(y), np.median(x),1, np.min(y)]
    popt, pcov, infodict, mesg, ier = curve_fit(_step_fun, x, y,p0=p0, full_output=True, maxfev = 1000000)
    step_size , location, stepness, b = popt
    
    info = StepInfo(step_size , int(np.floor(location)), stepness, b)
    return info

info = get_step_location_size(x_data, y_data)
print(info)
# location, step_size, stepness, b, fitted_data = get_step_location_size(x_data, y_data)
# print(f"location:{location}, step_size:{step_size}, stepness:{stepness}, b:{b}")
# plt.plot(x_data, fitted_data, '-y', label="Fitted data")

StepInfo(step_size=-67.74641142337659, location=348, stepness=76.16289667023504, b=220.7618434328121)


C:\Users\CellphyLab\AppData\Local\Temp\ipykernel_20148\1875578234.py:21: RuntimeWarning: overflow encountered in exp
  y = step_size / (1 + np.exp(-stepness*(x-location))) + b
C:\Users\CellphyLab\.conda\envs\seg4d\lib\site-packages\scipy\optimize\_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [23]:
steps = []
window_size_limit = 40
counter=0
windows_to_process = []

"""
def step_generator(x, y):
    info = get_step_location_size(x, y)
    steps.append(info)
    
    left_y = y[:info.location]
    left_x = x[:info.location]
    if len(left_y)>window_size_limit:
        windows_to_process.append((left_x, left_y))
        
    right_y = y[info.location+1:]
    right_x = x[info.location+1:]
    
    if len(right_y)>window_size_limit:
        windows_to_process.append((right_x, right_y))
        
step_generator(x_data, y_data)
"""
windows_to_process.append((x_data,y_data))

while len(windows_to_process):
    print(len(windows_to_process))
    p_x, p_y = windows_to_process.pop(0)
    
    info = get_step_location_size(p_x, p_y)
    steps.append(info)
    
    location_index = np.where(p_x == info.location)[0][0]
    
    left_x = p_x[:location_index]
    left_y = p_y[:location_index]
    if len(left_y)>window_size_limit:
        windows_to_process.append((left_x, left_y))
        
    right_x = p_x[location_index+1:]
    right_y = p_y[location_index+1:]
    if len(right_y)>window_size_limit:
        windows_to_process.append((right_x, right_y))


1
2
3
4
5
6
7
7


C:\Users\CellphyLab\AppData\Local\Temp\ipykernel_20148\1875578234.py:21: RuntimeWarning: overflow encountered in exp
  y = step_size / (1 + np.exp(-stepness*(x-location))) + b


8
9
8
7
6
6
6
5
5
6
5
4
4
4
3
2
1


In [24]:
steps

[StepInfo(step_size=-67.74641142337659, location=348, stepness=76.16289667023504, b=220.7618434328121),
 StepInfo(step_size=181.27826442555556, location=248, stepness=22.144786646604118, b=168.83693340899046),
 StepInfo(step_size=332.73476921910554, location=498, stepness=18.565000989266633, b=19.0132525613254),
 StepInfo(step_size=-213.7385702186162, location=98, stepness=16.23983663299247, b=297.3901814746867),
 StepInfo(step_size=-2.15069227506997, location=291, stepness=15.858889061792143, b=351.17332903199105),
 StepInfo(step_size=-8.277207647430492, location=421, stepness=15.525871689286811, b=23.235234240619036),
 StepInfo(step_size=-4.419435670272838, location=511, stepness=-27.966249983686538, b=352.3168597483539),
 StepInfo(step_size=-6.324756432409051, location=44, stepness=16.175741691028016, b=300.73188450406536),
 StepInfo(step_size=-50.790616647195044, location=198, stepness=1.1487181997973956, b=100.78618098563375),
 StepInfo(step_size=-8.872529523532506, location=275, 

In [25]:
steps.sort(key=lambda x: x.location)


In [26]:
steps

[StepInfo(step_size=9.050397365224308, location=21, stepness=24.527810066948096, b=295.90306725531144),
 StepInfo(step_size=-6.324756432409051, location=44, stepness=16.175741691028016, b=300.73188450406536),
 StepInfo(step_size=6.097699564674777, location=46, stepness=18.368735360774455, b=286.28420791853074),
 StepInfo(step_size=7.718116125482862, location=53, stepness=-16.43206597643009, b=291.20739069157963),
 StepInfo(step_size=30.725093375106933, location=93, stepness=11.166751155365992, b=292.0901164631009),
 StepInfo(step_size=-213.7385702186162, location=98, stepness=16.23983663299247, b=297.3901814746867),
 StepInfo(step_size=-8.351158631135739, location=121, stepness=17.904647844586787, b=105.74045438879138),
 StepInfo(step_size=1.3647646684711516, location=152, stepness=-15.177542296838428, b=99.84309178639522),
 StepInfo(step_size=10.682230670712238, location=166, stepness=17.900602708915887, b=92.48401832713671),
 StepInfo(step_size=-50.790616647195044, location=198, step